$group

In [ ]:
# Connection to pymongo 

In [7]:
mongo_url = 'mongodb://localhost:27017/'


In [8]:
from pymongo import MongoClient
client = MongoClient(mongo_url)
db = client.cloud_resources # database
inventory = db.inventory   # collection

# Example 1:

In [9]:
group_stage = {"$group": {"_id": "all", "sum": {"$sum": 1}}}

In [13]:
results = db.inventory.aggregate([group_stage])

In [14]:
unwrapped_records = [record for record in results]

In [15]:
unwrapped_records

[{'_id': 'all', 'sum': 1000000}]

# Example 2:

## Count command 

In [21]:
results = db.inventory.count_documents({})
#record_count = results.count_documents()


In [22]:
results

1000000

# Example 3:

In [23]:
group_avg_stage = {"$group": {"_id": "all", "avg": {"$avg": "$field"}}}
group_max_stage = {"$group": {"_id": "all", "max": {"$max": "$field"}}}

# Example 4:

## group the document by field 

In [24]:
group_by_field = {"_id": "$status", "total_count": {"$sum":  1}}

In [25]:
results = db.inventory.aggregate([group_stage])

In [26]:
unwrapped_records = [record for record in results]

In [27]:
unwrapped_records

[{'_id': 'all', 'sum': 1000000}]

# Example 5:

### group_by = {"_id": "$staus", "sum": {"$sum": 1}}

#### groupby is divided in two parts 

1. group by key 
2. measures

##### group_by_field = {"_id": "$status", "total_count.status_count": {"$sum":  1}}
Note the field name contain at . , this will lead to an error 

###### group_invalid_exp = {'_id': '$qty',  'stats': {"sum": {"$sum": 1}}, {"avg": {"$avg": "$sku"}}}

this will lead to an error because mongo expect direct aggregation after the field name 

### Value of field 
if you have a data in collection like 
```python
{"resource_name": {"name": "VPC Network", "value": "aws:3y2ui23:VPC"}}
```
and you want to do groupby ```name```

In [37]:
group_by_name = {"$group": {"_id": "$provider_resource_id.value", "sum": {"$sum": 1}}}

In [38]:
results = db.inventory.aggregate([group_by_name])
unwrapped_records = [record for record in results]
unwrapped_records

[{'_id': '7HG7727O4PHRPPM9BI2XJEWF3TJQ2RKQN3VN4CKZ1B4RF', 'sum': 4},
 {'_id': 'UHX29HQXOOBI4ZMBDRAH75WXZHK7FET8927KPPK3VQJ6X', 'sum': 4},
 {'_id': 'CLUZNX8FLPYQ94LHLGCY3NLPF8F14VZ537FI9BNSY7T24', 'sum': 1},
 {'_id': 'UVMDGBV64HTZSXMJXZBH9HXRB6V9I2DA6KPJG66EE0K40', 'sum': 6},
 {'_id': 'FEF6I1KICHMZYU87SVOF5RN8LGK0T4A52A57CTYESF76T', 'sum': 6},
 {'_id': 'NNV305DZIW8R065MOLTZERVOON0QEVDY96W2DM2W90LBZ', 'sum': 3},
 {'_id': 'KBAJ9R86SGFUY4JB2NOOFWQ4U8EA1XPI8JBHMLER3EX45', 'sum': 1},
 {'_id': 'EBC2N6USGGEH9FDCQV40WF8UUABHREX2QX6UR5TVW1IIP', 'sum': 1},
 {'_id': 'SV8LUSZBCL2W7F66E38FCU55APMQGCGEEPWKUCH25DS4H', 'sum': 1},
 {'_id': 'KSU6TCEJB34DUCRHK7612LMNQVJ72R4IWG84XFOCP36N2', 'sum': 2},
 {'_id': 'RMDY5C765ACSW2YKD3OAQRPZ91DSKOXL0SSRCQR515TML', 'sum': 2},
 {'_id': 'H1MX81UVDBMJMDW3874RLCNRRZ5JER2HC0MNO52A03EM8', 'sum': 1},
 {'_id': 'M3RO8ZZ270P8RY61SOOWM978A7IMIPJ9UTHW7BKXD4UTB', 'sum': 2},
 {'_id': 'VO7ZFME6G5MX6NC03705K11NAD5LADY2O8395PTO6UTWX', 'sum': 5},
 {'_id': 'L78JYFKOX08THDS9821I8UIE

## Example 6:


#### $addToSet

##### Purpose of addToset : when we do groupby id 'field_name', and if we want to get the all distinct values in that group.

#### example: group the data by type of resource and to which all provider these records are attached.

In [44]:
group_addtoset = {"$group": {"_id": "$type", "provider": {"$addToSet": "$provider_code"}}}

In [45]:
results = db.inventory.aggregate([group_addtoset])
unwrapped_records = [record for record in results]
unwrapped_records

[{'_id': 'independent_component', 'provider': ['aws', 'TestProvider']},
 {'_id': 'service_instance', 'provider': ['aws', 'TestProvider']}]

### groupby region code

In [46]:
group_region_sum = {"$group": {"_id": "$location.region_code", "sum": {"$sum": 1}}}

In [47]:
results = db.inventory.aggregate([group_region_sum])
unwrapped_records = [record for record in results]
unwrapped_records

[{'_id': 'ca-central-1', 'sum': 52634},
 {'_id': 'eu-west-1', 'sum': 52474},
 {'_id': 'ap-southeast-2', 'sum': 52178},
 {'_id': 'eu-west-2', 'sum': 52951},
 {'_id': 'ap-northeast-1', 'sum': 53005},
 {'_id': 'cn-north-1', 'sum': 52528},
 {'_id': 'sa-east-1', 'sum': 52099},
 {'_id': 'cn-northwest-1', 'sum': 52731},
 {'_id': 'eu-central-1', 'sum': 52055},
 {'_id': 'eu-west-3', 'sum': 53044},
 {'_id': 'us-east-1', 'sum': 53185},
 {'_id': 'ap-south-1', 'sum': 52716},
 {'_id': 'us-west-1', 'sum': 52245},
 {'_id': 'US', 'sum': 52789},
 {'_id': 'us-east-2', 'sum': 52432},
 {'_id': 'ap-southeast-1', 'sum': 53054},
 {'_id': 'ap-northeast-2', 'sum': 52319},
 {'_id': 'ap-northeast-3', 'sum': 52688},
 {'_id': 'us-west-2', 'sum': 52873}]

#### If we want to accumulate the items even the duplicate ones

### $push

In [51]:
group_push = {"$group": {"_id": "$type", "provider": {"$push": "$provider_code"}}}

In [52]:
results = db.inventory.aggregate([group_push])
unwrapped_records = [record for record in results]
unwrapped_records

[{'_id': 'independent_component',
  'provider': ['TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   'TestProvider',
   '

#### $first

In [55]:
group_first_sample = {"$group": {"_id": "$type", "sample": {"$first": "$provider_code"}}}

In [56]:
results = db.inventory.aggregate([group_first_sample])
unwrapped_records = [record for record in results]
unwrapped_records

[{'_id': 'independent_component', 'sample': 'TestProvider'},
 {'_id': 'service_instance', 'sample': 'TestProvider'}]

### $last

In [61]:
group_last_sample = {"$group": {"_id": "$type", "sample_resource_id": {"$last": "$provider_resource_id"}}}

In [64]:


import pprint
pprint.pprint(list(db.inventory.aggregate([group_last_sample])))

[{'_id': 'independent_component',
  'sample_resource_id': {'name': 'FS9K5EZ4W76VW15WEPFR0V137LSXL0ZWEYUFHJX22UP1X5URU0P8XLU33C4SBS6FM',
                         'value': 'LE60ZQAGXBYD87KADHUZJG6IMF4FQ79LNX927TBYT1XMI'}},
 {'_id': 'service_instance',
  'sample_resource_id': {'name': 'JOIJCFIZ5BFJITFQRTCM73DD01MJOL8XURUGSNWUJDG9ZRFF2F4LVZAI05Q9GQ549',
                         'value': 'N5MW58GMADRD4OWZ3AIY0FXO4JE0KLVVQUXMWUOZQ0M19'}}]


### Run the explain plan

In [66]:
db.command('aggregate', 'inventory', pipeline=[group_last_sample], explain=True)

{'stages': [{'$cursor': {'query': {},
    'fields': {'provider_resource_id': 1, 'type': 1, '_id': 0},
    'queryPlanner': {'plannerVersion': 1,
     'namespace': 'cloud_resources.inventory',
     'indexFilterSet': False,
     'parsedQuery': {},
     'winningPlan': {'stage': 'COLLSCAN', 'direction': 'forward'},
     'rejectedPlans': []}}},
  {'$group': {'_id': '$type',
    'sample_resource_id': {'$last': '$provider_resource_id'}}}],
 'ok': 1.0}

### Arbitrary sample

```
$first : First record encountered in the pipeline
$last : last record encountered in the pipeline
    ```

```
by default: mongo pass document in pipeline  acc to the ascending _id 

if you want predictable first and last record , order of document is required in pipeline
```